# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read CSV and save to dataframe
city_data = pd.read_csv(f"../WeatherPy/output_data/cities.csv")
city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ekhabi,53.5102,142.9671,31.59,83,100,5.50,RU,1619740220
1,plettenberg bay,-34.0527,23.3716,56.89,84,89,5.88,ZA,1619739648
2,rongcheng,30.6340,117.8517,71.10,67,0,5.39,CN,1619740221
3,rikitea,-23.1203,-134.9692,76.44,67,89,10.69,PF,1619740221
4,zarand,30.8127,56.5640,67.46,38,100,2.66,IR,1619740222
...,...,...,...,...,...,...,...,...,...
552,najran,17.4924,44.1277,73.40,56,75,2.30,SA,1619740160
553,walvis bay,-22.9575,14.5053,55.99,82,0,4.61,NaN,1619740386
554,ucluelet,48.9329,-125.5528,54.00,87,90,12.66,CA,1619740386
555,tabou,4.4230,-7.3528,80.10,81,100,5.06,CI,1619740387


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
city_data = city_data.dropna()

# Set locations
locations = city_data[["Lat", "Lng"]].astype(float)

# Convert humidity to float and store
humidity = city_data["Humidity"].astype(float)

In [5]:
# Create a poverty Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2.5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# My conditions: 
    # Temperature between 68 and 86 Farenheit
    # Cloudiness of 40% or less
    # Humidity between 30% and 50%
cond1 = city_data[city_data['Max Temp'] <= 86]
cond1 = cond1[cond1['Max Temp'] >= 68]
cond2 = cond1[cond1['Cloudiness'] <= 40]
cond3 = cond2[cond2['Humidity'] <= 50]
cities_criteria = cond3[cond3['Humidity'] >= 30]
cities_criteria

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,sadiqabad,28.3062,70.1307,83.57,30,32,7.34,PK,1619740226
106,port lincoln,-34.7333,135.8667,68.47,44,28,11.25,AU,1619740250
123,abu samrah,35.3029,37.1841,69.58,43,2,3.18,SY,1619740254
131,turayf,31.6725,38.6637,69.80,30,0,11.50,SA,1619739791
289,riyadh,24.6877,46.7219,73.40,35,0,7.11,SA,1619739516
331,tura,25.5198,90.2201,77.18,41,4,3.94,IN,1619740315
410,port hedland,-20.3167,118.5667,75.20,50,0,3.44,AU,1619740339
443,platteville,42.7342,-90.4785,71.60,45,20,13.80,US,1619740350
446,morro bay,35.3658,-120.8499,82.00,31,17,1.99,US,1619740300
472,oyama,36.3000,139.8000,69.01,48,20,11.50,JP,1619740360


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Get cities coordinates and look for airports
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
cities_hotels = cities_criteria[['City','Lat','Lng']]
hotels = []
for index, row in cities_hotels.iterrows():
    lats = cities_hotels['Lat'][index]
    lngs = cities_hotels['Lng'][index]
    params = {
        "location": f"{lats},{lngs}",  
        "rankby": "distance",
        "keyword": "hotel",
        "key": g_key,}
    # Get airports
    response = requests.get(base_url, params=params).json()
    try:
        results = response['results']
        hotels.append(results[0]['name'])
    except:
        print(f"A nearby hotel was not found for {cities_criteria['City'][index]}")
        cities_criteria = cities_criteria.drop(index)
hotels

A nearby hotel was not found for abu samrah


['Iqra Guest House',
 'Marina Park View',
 'Orchida Turaif Hotel Suites 1',
 'Rose garden hotel',
 'Hotel RIKMAN Continental',
 'The Pier Hotel',
 'Holiday Inn Express & Suites Platteville',
 'The Landing at Morro Bay',
 'Oyama Palace Hotel',
 'The Westin Annapolis',
 'Dungsam Trashiling Resort',
 'نادي و فندق القوات المسلحة']

In [8]:
# Add to dataframe
hotel_df = pd.DataFrame(cities_criteria)
hotel_df['Hotel Name'] = hotels
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
20,sadiqabad,28.3062,70.1307,83.57,30,32,7.34,PK,1619740226,Iqra Guest House
106,port lincoln,-34.7333,135.8667,68.47,44,28,11.25,AU,1619740250,Marina Park View
131,turayf,31.6725,38.6637,69.80,30,0,11.50,SA,1619739791,Orchida Turaif Hotel Suites 1
289,riyadh,24.6877,46.7219,73.40,35,0,7.11,SA,1619739516,Rose garden hotel
331,tura,25.5198,90.2201,77.18,41,4,3.94,IN,1619740315,Hotel RIKMAN Continental
410,port hedland,-20.3167,118.5667,75.20,50,0,3.44,AU,1619740339,The Pier Hotel
443,platteville,42.7342,-90.4785,71.60,45,20,13.80,US,1619740350,Holiday Inn Express & Suites Platteville
446,morro bay,35.3658,-120.8499,82.00,31,17,1.99,US,1619740300,The Landing at Morro Bay
472,oyama,36.3000,139.8000,69.01,48,20,11.50,JP,1619740360,Oyama Palace Hotel
502,mayo,38.8876,-76.5119,80.60,50,40,13.18,US,1619740080,The Westin Annapolis


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations, 
    info_box_content=[f"Name: {hotel_df['Hotel Name'][index]} | City: {hotel_df['City'][index]} | Country: {hotel_df['Country'][index]}"
                      for index, row in hotel_df.iterrows()]
)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))